In [2]:
%matplotlib inline
import os
os.chdir('U:\\GitHubClones\\SALTSeniorDesign')
os.getcwd()
#import Excel
import xlwings as xw
import seaborn as sb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import re
from datetime import datetime
import datetime
import statsmodels.formula.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn import preprocessing
import time

In [3]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
cols = ['assetNumber', 'reportingPeriodEndingDate','reportingPeriodBeginningDate',
        'originalLoanTerm','originalLoanAmount','vehicleValueAmount','originalInterestRatePercentage','subvented',
        'vehicleNewUsedCode', 'vehicleModelYear','vehicleTypeCode',
        'obligorCreditScore','obligorIncomeVerificationLevelCode', 'obligorEmploymentVerificationCode','obligorGeographicLocation',
        'paymentToIncomePercentage', 'reportingPeriodActualEndBalanceAmount',
        'reportingPeriodBeginningLoanBalanceAmount','totalActualAmountPaid',
        'remainingTermToMaturityNumber',
        'currentDelinquencyStatus','coObligorIndicator',
        'zeroBalanceCode']

logitRemoveCols = ['assetNumber', 'reportingPeriodEndingDate','originalLoanAmount','vehicleValueAmount','reportingPeriodActualEndBalanceAmount',
        'reportingPeriodBeginningLoanBalanceAmount','totalActualAmountPaid','vehicleNewUsedCode','zeroBalanceCode',
        'vehicleModelYear','currentDelinquencyStatus','subvented','obligorIncomeVerificationLevelCode', 'obligorEmploymentVerificationCode',
        'obligorGeographicLocation','reportingPeriodBeginningDate']

In [82]:
wb = xw.Book('Salt.xlsm')
setup = wb.sheets('Setup')
output = wb.sheets('Output')
strats = wb.sheets('Strat Tables')
performance = wb.sheets('Performance')

# Path For the Current deal that is selected based on Setup
directory = setup['Setup.Directory'].value
shelf = setup['Setup.Shelf'].value
shelfList = setup['Shelf_List'].value
deal = setup['Setup.Deal'].value
report = setup['Setup.Report'].value
num_Deals = setup['Setup.Num_Deals'].value
max_Report = setup['Setup.Max_Report_Num'].value


In [96]:
tapes = np.array(setup['Tapes'].value)
tapes = tapes[0:int(max_Report),0:int(num_Deals)]
dfTapes = pd.DataFrame(data=tapes[1:,1:], columns=tapes[0,1:])
dealList = list(dfTapes.columns)

In [94]:
def chargeOffs(trainOrTest, tapes, dealList, shelfList, unemploymentPath):
    shelfList = list(filter(lambda a: a != 'None', shelfList))
    train = pd.DataFrame()
    for k in range(3, len(dealList)):
        
        monthList = list(tapes[dealList[k]].astype(str))
        monthList = list(filter(lambda a: a != 'None', monthList))
        monthList = monthList[:-1]
        
        if trainOrTest == 'Test':
            monthList = [monthList[-1]]
        
        for i in range(0, len(monthList)):
            path = directory + '\\' + shelfList[k] + '\\' + dealList[k] + '\\' + monthList[i]
            print(dealList[k])
            df = pd.read_csv(path, usecols=cols, memory_map=True, low_memory=False)
            df = df[~df.zeroBalanceCode.isin(['-'])]#taking out all of the performing loans so far

            df['LTV'] = (df['originalLoanAmount']/df['vehicleValueAmount'])*100
            df['loanDiff'] = (df['originalLoanAmount'] - df['reportingPeriodBeginningLoanBalanceAmount'])
            df['coObligorIndicator'] = df['coObligorIndicator'].astype(int)

            umrate = pd.read_csv(unemploymentPath, memory_map=True, low_memory=False)
            umrate['Date'] = pd.to_datetime(umrate.Date)
            umrate['Date'] = umrate['Date'].dt.strftime('%m/%d/%Y')
            umrate = umrate.set_index('Date')
            stateDict = umrate.to_dict(orient='index')

            df['reportingPeriodBeginningDate'] = pd.to_datetime(df['reportingPeriodBeginningDate']).dt.strftime('%m/%d/%Y')
            states = list(df['obligorGeographicLocation'])
            uniqueList = df.reportingPeriodBeginningDate.unique()
            umList = []
            for j in range(0, len(states)):
                if not uniqueList[0]:
                    continue
                val = stateDict[uniqueList[0]].get(states[j])
                umList.append(val)
            unemployment = pd.Series(umList)
            df = df.assign(StateUnemployment=unemployment.values)
            train = train.append(df)
    
    
    return train

In [95]:
umRatePath = 'U:\\GitHubClones\\SALTSeniorDesign\\' + 'State_level_unemployment_rate.csv'
dfTrain = chargeOffs('Test', dfTapes, dealList, shelfList, umRatePath)

dfTrain.to_csv('AggregateChargeOffsTest.csv')

DRIVE 2017-1
DRIVE 2017-2
DRIVE 2017-3
SDART 2017-1
SDART 2017-2
SDART 2017-3
